In [ ]:
#Save con Nodos
import os
from keras.datasets import cifar100,cifar10,mnist,fashion_mnist
from keras.models import load_model
import itertools
from matplotlib import pyplot as plt
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import itertools
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from tqdm import tqdm_notebook
from keras.regularizers import l2
from keras.activations import tanh
from keras.models import Sequential
from keras.layers import Dense, Flatten,Conv2D, MaxPooling2D,Conv2D, MaxPooling2D,Dropout,AveragePooling2D,GlobalAveragePooling2D,Activation
from keras.losses import categorical_crossentropy,binary_crossentropy
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras import optimizers
from keras import backend as K
from time import time
import random
from random import shuffle,seed
from sklearn.metrics import classification_report
import math
from PIL import Image
from time import time
import numpy
import tensorflow
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Dropout, Flatten, Dense, Lambda
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
import theano.tensor as T
from keras.callbacks import EarlyStopping
from keras.backend import eval


model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(15,15,1), strides=(1, 1),activation="relu", padding="same"))
model.add(MaxPooling2D((2,2), strides=(1,1)))
model.add(Dropout(0.1))
model.add(Conv2D(128, (5, 5), strides=(1, 1), activation="relu", padding="same"))
model.add(Conv2D(256, (5, 5), strides=(1, 1), activation="relu", padding="same"))
#model.add(Flatten())
model.add(GlobalAveragePooling2D())
model.add(Dense(512, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, kernel_initializer='random_uniform',activation='relu'))
model.add(Dense(1, kernel_initializer='random_uniform',activation='softmax'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.05, momentum=0.9),metrics=['accuracy'])

In [ ]:
#test con random
import random
x_train = []
for i in range(100000):
    x_train.append(np.random.rand(15,15))
    
x_train = np.asarray(x_train)

#y_train = np.asarray([1,1,0,1,1,1,1,0,0,1])
x_train = np.expand_dims(x_train, axis=4)
x_train.shape
  
y_train = np.asarray([0]*50000 + [1]*50000)
random.shuffle(y_train)

print(y_train.shape)
print(x_train.shape)

In [ ]:
model.fit(x_train, y_train, batch_size=100, epochs=350, verbose=1)

# Segundo Modelo

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(15,15,1), strides=(1, 1),activation="relu", padding="same"))
model.add(Dropout(0.1))
model.add(GlobalAveragePooling2D())
model.add(Dense(128, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer='random_uniform',activation='relu'))
model.add(Dense(1, kernel_initializer='random_uniform',activation='softmax'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.05, momentum=0.9),metrics=['accuracy'])